In [1]:
import numpy as np
import torch
import torch.nn as nn 
import torch.nn.functional as F
import pandas as pd
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('../../plant_pathology/scripts/')
from dataset.dataloaders import GetLoaders
from dataset.dataset import ReadCSV
import os
from train.loss import loss_fn
from model.resnet32 import get_model
from model.vgg16 import get_vgg16
from train.train_script import train, eval_dataset
import torchvision
from dataset.resize_images import resize_images
import glob
# from kaggle.api.kaggle_api_extended import KaggleApi
import pickle
from metrics.metrics import generate_metrics
import matplotlib.pyplot as plt
import torchvision

In [2]:
temp = ReadCSV('../../plant_path_data/train.csv', '../../plant_path_data/train_images_224/')
temp.read_csv()
temp.check_paths()
temp.encode_labels()
temp.one_hot_encoder()
temp.split_datasets()

In [15]:
data_object = temp.get_object()
loader_object = GetLoaders(data_object.train, data_object.val, data_object.test)
loader_object.create_sets()
loader_object.create_loaders(shuffle_train=False, batch_size=256)
train_loader, val_loader, test_loader = loader_object.return_loaders()

In [16]:
loss, preds = eval_dataset(get_model(12), train_loader, weights_path='../../weights/res50_horizontal_flip_random_brightness_contrast_smoothing_v1/47.pth', return_preds = True)

In [28]:
preds_updated = torch.argmax(F.softmax(preds), dim=1).int()

In [31]:
labels = torch.Tensor(train_loader.dataset.labels).int()

In [48]:
wrong_preds = torch.where(preds_updated != labels)[0]

In [52]:
wrong_preds_paths = np.array(train_loader.dataset.paths)[wrong_preds]

In [56]:
# 